### Testing the `PuLP` library
*https://coin-or.github.io/pulp/index.html*

In [2]:
from pulp import *

In [6]:
prob = LpProblem("TheWhiskasProblem", LpMinimize);

In [7]:
x1 = LpVariable("ChickenPercent", 0, None, LpInteger)
x2 = LpVariable("BeefPercent", 0)

In [14]:
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredient per can"

In [16]:
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.1 * x1 + 0.2 * x2 >= 8.0, "ProteinRequirement"
prob += 0.08 * x1 + 0.1 * x2 >= 6.0, "FatRequrement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequirement"

In [18]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent]

In [19]:
prob.solve()

1

In [20]:
print(f"Status: {LpStatus[prob.status]}")

Status: Optimal


In [23]:
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

BeefPercent = 66.0
ChickenPercent = 34.0


In [24]:
print(f"Total Cost of Ingredient per can = {value(prob.objective)}")

Total Cost of Ingredient per can = 0.97


### The problem

Consider an airline that has demand for baggage handlers in a domesti airport. THe hourly requrements for 24 hours starting from midnihgt are 5, 4, 3, 2, 2, 5 6, 7, 9, 11, 13, 14, 11, 9, 11, 12, 14, 15, 18, 20, 16, 10, 7 and 5 respectively. There are 6 shifts for handlers: the first shift may start at midnight, at 1AM, at 2AM or at 3AM. If the first shift started at hour $H$, the next shifts start at $H + 4$, $H + 8$, $H + 12$, $H + 16$ and $H + 20$. Each shift wroks for 9 ocnsecutive hours. The demand is the same for all days. Using an integer programming model find the minimum number of handlers required that meets the demand and hour at which the first shift should start so that this minimum number of handlers is minimized.

#### Solution

Let us denote number of handlers at $i$-th shift by $x_i$, where $i \in \{ 1, \dots, 6 \}$. Then our problem is to minimize the total number of handlers, i.e.

$$
\min \sum_{i=1}^6 x_i
\text.
$$

We will have four different optimization problems depending on the shift start time. For example, if the shift would start at midnight, then for first eight hours we would have

\begin{array}{}
x_1 + x_6 \ge 5 \\
x_1 + x_6 \ge 4 \\
x_1 + x_6 \ge 3 \\
x_1 + x_6 \ge 2 \\
x_1 + x_2 \ge 2 \\
x_1 + x_2 \ge 5 \\
x_1 + x_2 \ge 6 \\
x_1 + x_2 \ge 7 \\
\qquad\dots
\end{array}

And if the shift would start at 1AM, then the handlers would shift one row down, i.e.

\begin{array}{}
x_5 + x_6 \ge 5 \\
x_1 + x_6 \ge 4 \\
x_1 + x_6 \ge 3 \\
x_1 + x_6 \ge 2 \\
x_1 + x_6 \ge 2 \\
x_1 + x_2 \ge 5 \\
x_1 + x_2 \ge 6 \\
x_1 + x_2 \ge 7 \\
\qquad\dots
\end{array}

Each of those problem can be also presented in a matrix form, for example $A_{12AM}$ would be a $24$ rows by $6$ columns matrix. Where each row would have only two nonzero values corresponding to the current baggage handlers working. 

Then the mimimization problem would be

\begin{array}{}
\min & c^\intercal x \\
\text{1subject to} & A_{12AM} x > b \\
\end{array}

where $c \in \mathbf{R}^6$ is a vector of ones and $b \in \mathbf{R}^{24}$ is a vector of corresponding hourly requirements.

We thus have four problems to solve:

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{12AM} x \ge b \\
\tag{1}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{1AM} x \ge b \\
\tag{2}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{2AM} x \ge b \\
\tag{3}
\end{array}

\begin{array}{}
\min & c^\intercal x \\
\text{subject to} & A_{3AM} x \ge b \\
\tag{4}
\end{array}

Where only the matrices are different. From the obtained solutions we can manually choose the optimal one. Although the problems are not in the cannonical form (we would have to switch the sign in the inequality), the software we will be using does not require that.

#### Implementation

In [28]:
# Not yet done

In [25]:
hourly_requirements = [5, 4, 3, 2, 2, 5, 6, 7, 9, 11, 13, 14, 11,
                       9, 11, 12, 14, 15, 18, 20, 16, 10, 7, 5]